In [70]:
import pandas as pd
from datetime import datetime
import re

In [71]:
def open_file(filename):
    f = open(filename, "r")
    text = f.read()
    lines = text.split('\n')
    return lines

def open_file(filename):
    f = open(filename, "r")
    text = f.read()
    lines = text.split('\n')
    return lines


def clean_name(name: str) -> str:
    char_add_ons = ['V.O.', "'S COM VOICE",
                    'Cont\'d', 'O.S.', "'S", "VOICE", "INTERCOM"]
    for txt in char_add_ons:
        if txt in name:
            name = name.replace(txt, '')
    return re.sub("[\(\[].*?[\)\]]", "", name.strip())


def get_title(lines):
    series_ind = 0
    for i in range(len(lines)):
        if "STAR TREK: THE NEXT GENERATION" in lines[i] or 'STAR TREK: DEEP SPACE NINE' in lines[i]:
            series_ind = i
            break
    return str(lines[series_ind+2]).strip().replace('"','')


def get_date(lines):
    draft_ind = 0
    for i in range(len(lines)):
        if "FINAL DRAFT" in lines[i]:
            series_ind = i
            break
    try:
        date = str(lines[series_ind+2])
        date = date.strip()
        final = datetime.strptime(date, '%B %d, %Y')
    except:
        print(lines[series_ind+2])
        final = "AHHH"
    return final



In [72]:
def get_quotes(lines):
#    characters = [x for x in lines if '\t\t\t\t\t' in x]
    char_index = [i for i in range(len(lines)) if '\t\t\t\t\t' in lines[i]]
    scenes_index = [j for j in range(len(lines)) if 'INT.' in lines[j] or 'EXT.' in lines[j]]

    data = []

    for i in char_index:
        name = lines[i].replace('\t\t\t\t\t', '')
        if "FADE OUT" in name:
            continue

        j = i+1
        next_line = lines[j]

        quote = ""

        while next_line != '':
            next_line = next_line.replace('\t', ' ')
            quote += re.sub("[\(\[].*?[\)\]]", "", next_line)
            j = j+1
            next_line = lines[j]

        scene = lines[max([x for x in scenes_index if x < i])]

        data.append([name, quote,scene])

    test_df = pd.DataFrame(data, columns=['character', 'quote','scene'])
    return (test_df)

def clean_location(loc:str):
    views = ['INT.','EXT.']
    output = loc
    for v in views:
        if v in output:
            output = output[output.index(v)+5:]
    if '(' in output:
        output = output[:output.index('(')-1]
    elif '-' in output:
        output = output[:output.index('-')-1]
    # for '(' in other:
    #     if c in output:
    #         output = output[:output.index(c)-1]
    return(output)

def get_view(loc:str):
    output = None
    views = ['INT.','EXT.']
    for v in views:
        if v in loc:
            output=v
    return(output)


def create_df(filepath: str) -> pd.DataFrame:
    lines = open_file(filepath)
    test_df = get_quotes(lines)
    test_df['location']=test_df['scene'].apply(clean_location)
    test_df['view']=test_df['scene'].apply(get_view)
    test_df['character'] = test_df['character'].apply(clean_name)
    test_df['episode'] = get_title(lines)
    return test_df


In [73]:
create_df('scripts_ds9/408.txt')

,character,quote,scene,location,view,episode
0,SISKO,Station log. Stardate 46910.1. Chief O...,3A EXT. SPACE - DEEP SPACE NINE (OPTICAL),SPACE - DEEP SPACE NINE,EXT.,Dax
1,BASHIR,It's almost as though the re- filtration ...,4 INT. REPLIMAT,REPLIMAT,INT.,Dax
2,DAX,The technology looks like something the ...,4 INT. REPLIMAT,REPLIMAT,INT.,Dax
3,BASHIR,"(bored, trying to be clever) Perhaps...",4 INT. REPLIMAT,REPLIMAT,INT.,Dax
4,DAX,Or is it an R-C-L type two...,4 INT. REPLIMAT,REPLIMAT,INT.,Dax
...,...,...,...,...,...,...
370,BASHIR,Are you all right?,98A INT. PROMENADE - LOWER LEVEL,PROMENADE,INT.,Dax
371,DAX,"Yes. As a matter of fact, I think I ...",98A INT. PROMENADE - LOWER LEVEL,PROMENADE,INT.,Dax
372,BASHIR,That was four days ago.,98A INT. PROMENADE - LOWER LEVEL,PROMENADE,INT.,Dax
373,DAX,Was there a time limit?,98A INT. PROMENADE - LOWER LEVEL,PROMENADE,INT.,Dax


In [74]:
lines = open_file('102.txt')
x=get_quotes(lines)
t = x['scene'].unique()
t

array(['1    EXT. SPACE - STARSHIP (OPTICAL)', '3    INT. ENGINE ROOM',
       '5    INT. LOUNGE DECK', '6    INT. BRIDGE - WIDE ANGLE',
       '10   INT. BRIDGE - VARIOUS ANGLES',
       '15   EXT. U.S.S. ENTERPRISE AND GRID (OPTICAL)',
       '27   INT. BRIDGE - ANGLES INCLUDING MAIN VIEWER',
       '30   INT. BRIDGE - VARIOUS ANGLES',
       '34   EXT. SPACE - THE ENTERPRISE (OPTICAL)',
       '35   INT. BATTLE BRIDGE', '36   INT. MONTAGE OF SAUCER SETS',
       '37   INT. BATTLE BRIDGE - VARIOUS ANGLES',
       '38   INT. BRIDGE - EMPHASIZING LT. WORF',
       '40   INT. BATTLE BRIDGE',
       '41   EXT. ENTERPRISE - ANGLES ON SEPARATION MECHANISM (OPTICAL))',
       '42   INT. BATTLE BRIDGE - INCLUDING MAIN VIEWER',
       '46   INT. "Q" COURTROOM - EMPHASIZING PICARD, DATA, TROI, TASHA',
       '56   INT. "Q" COURTROOM - WIDE ANGLE', '63   INT. BATTLE BRIDGE',
       '65   EXT. SPACE - LONG SHOT - PLANET (OPTICAL)',
       '68   INT. OLD CITY CORRIDOR - PANNING RIKER',
       "69

In [75]:
# INT & EXT might hint at scene